In [1]:
import os
import sys
sys.path.append('../')
# sys.path.append('../..')

In [2]:
from models.ollama_model_calls import query_phi, query_solar, query_starling, query_mistral
from utils.format_llm_output_text import format_llm_answer
from eval.gsm8k_llm import check_answer_using_llm, get_GSM8K_QA

In [4]:
from langchain_community.chat_models import ChatOpenAI

In [5]:
from typing import Optional

In [6]:
# OPENROUTER_API_KEY = os.environ['OPENROUTER_API_KEY']

class ChatOpenRouter(ChatOpenAI):
    openai_api_base: str
    openai_api_key: str
    model_name: str

    def __init__(self,
                 model_name: str,
                 openai_api_key: Optional[str] = None,
                 openai_api_base: str = "https://openrouter.ai/api/v1",
                 **kwargs):
        openai_api_key = openai_api_key or os.getenv('OPENROUTER_API_KEY')
        super().__init__(openai_api_base=openai_api_base,
                         openai_api_key=openai_api_key,
                         model_name=model_name, **kwargs)

In [7]:
from langchain_core.prompts import ChatPromptTemplate

In [8]:
llm = ChatOpenRouter(
    # model_name="mistralai/mistral-7b-instruct:free",
    model_name="mistralai/mixtral-8x7b-instruct",
    # model_name="fireworks/mixtral-8x22b-instruct-preview",
    # model_name="cognitivecomputations/dolphin-mixtral-8x7b"
)

In [9]:
# prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
# openrouter_chain = prompt | llm
# print(openrouter_chain.invoke({"topic": "banana"}))

# LangChain 

In [10]:
from langchain.agents import tool

In [11]:
import numexpr as ne

In [12]:
@tool
def Calculator(expression : str) -> int: # the name of the function takes the place of the name variable.
    """Use this tool when you need to math. NOTE: this tool uses python numexpr library, so format your input correctly please."""
    return ne.evaluate(expression).item()

In [13]:
tools = [Calculator]

In [14]:
from langchain.memory import ConversationBufferWindowMemory

In [15]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True,
    output_key="output",
)

In [16]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [17]:
# # Mistral compatability, change depending on LLM

# # Openhermes prompt format tokens
B_INST, E_INST = "<|im_start|>user\n", "<|im_end|>\n" #Instruction begin and end
B_SYS, E_SYS = "<|im_start|>system\n", "<|im_end|>\n" #System message begin and end

In [18]:
system = '''Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation'''

# system = B_SYS + '''Respond to the human as helpfully and accurately as possible. You have access to the following tools:

# {tools}

# Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

# Valid "action" values: "Final Answer" or {tool_names}

# Provide only ONE action per $JSON_BLOB, as shown:

# ```
# {{
#   "action": $TOOL_NAME,
#   "action_input": $INPUT
# }}
# ```

# Follow this format:

# Question: input question to answer
# Thought: consider previous and subsequent steps
# Action:
# ```
# $JSON_BLOB
# ```
# Observation: action result
# ... (repeat Thought/Action/Observation N times)
# Thought: I know what to respond
# Action:
# ```
# {{
#   "action": "Final Answer",
#   "action_input": "Final response to human"
# }}

# Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation''' + E_SYS

In [19]:
human = '''{input}

{agent_scratchpad}

(reminder to respond in a JSON blob no matter what)'''

# human = B_INST + '''\nUser: {input}

# {agent_scratchpad}

# (reminder to respond in a JSON blob no matter what)''' + E_INST

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human),
    ]
)

In [25]:
from langchain.agents import create_structured_chat_agent

In [26]:
agent = create_structured_chat_agent(llm=llm,
                                     tools=tools,
                                     prompt=prompt,)

In [27]:
from langchain.agents import AgentExecutor

In [28]:
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               handle_parsing_errors=True)

In [102]:
query = "Can you tell me what the total money I have if I have earned 544430GBP per day for 530 days?"
# answer = 288'547'900

In [103]:
# numexpr
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Question: Can you tell me what the total money I have if I have earned 544430GBP per day for 530 days?
Thought: To find the total money earned, I need to multiply the daily earnings by the number of days. I will use the Calculator tool for this calculation.
Action:
```
{
  "action": "Calculator",
  "action_input": "544430 * 530"
}
```
288547900```
{
  "action": "Final Answer",
  "action_input": "288547900 GBP"
}
```

> Finished chain.


{'input': 'Can you tell me what the total money I have if I have earned 544430GBP per day for 530 days?',
 'output': '288547900 GBP'}

In [29]:
email_context = "Monday had 20000 packages and a 20% MAPE. Tuesday had 40000 packages and a 15%  MAPE. Wednesday had 5000 packages and a 5% MAPE. "
question = "What is the WMAPE (weighted average by the package volume) for these days?"

query = email_context + question

agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
Thought: In order to calculate the WMAPE, I need to find the weighted average of the MAPE of each day. I can calculate this by multiplying the MAPE of each day by the proportion of packages delivered on that day, and then summing those values. To do this, I first need to find the total number of packages delivered over the three days, and then find the proportion of packages delivered on each day.

Action:
```
{
  "action": "Calculator",
  "action_input": "20000 + 40000 + 5000"
}
```
65000
Action:
```
{
  "action": "Calculator",
  "action_input": "20000 / 65000 * 20 + 40000 / 65000 * 15 + 5000 / 65000 * 5"
}
```
15.769230769230772
Action:
```
{
  "action": "Final Answer",
  "action_input": "The WMAPE for these days is approximately 15.77%."
}
```

> Finished chain.


{'input': 'Monday had 20000 packages and a 20% MAPE. Tuesday had 40000 packages and a 15%  MAPE. Wednesday had 5000 packages and a 5% MAPE. What is the WMAPE (weighted average by the package volume) for these days?',
 'output': 'The WMAPE for these days is approximately 15.77%.'}

the answer to  the WAPE question is: 0.1576923076923077

In [31]:
0.000217 + 0.000195 +0.000169 + 0.000165 + 0.000149

0.000895

^ the cost for running Mixtral this simple calc Q via openrouter: $0.000895 -> $0.001

https://openrouter.ai/activity